# 🧯 Příklad: Jednoduché kyvadlo s tlumením

Představme si:

- bodové těleso o hmotnosti \( m \),
- zavěšené na tuhé tyči délky \( l \),
- na kyvadlo působí:
  - gravitační moment \( -mgl \sin(\theta) \),
  - tlumicí moment \( -b \dot{\theta} \).

---

## ⚙️ 1. Rovnice pohybu

Z druhého Newtonova zákona pro rotaci:

$$
J \cdot \ddot{\theta} = \sum M
$$

Moment setrvačnosti bodového kyvadla:

$$
J = ml^2
$$

Působící momenty:

- gravitace: \( -mgl \sin(\theta) \),
- tlumení: \( -b \dot{\theta} \)

Dohromady:

$$
ml^2 \cdot \ddot{\theta} = -mgl \sin(\theta) - b \dot{\theta}
$$

Zkrátíme \( m \) a jedno \( l \):

$$
\ddot{\theta} + \frac{b}{ml^2} \dot{\theta} + \frac{g}{l} \sin(\theta) = 0
$$

---

## 🔁 2. Zavedeme stavové proměnné

Označíme:

- \( x_1 = \theta \) – úhel vychýlení,
- \( x_2 = \dot{\theta} \) – úhlová rychlost.

Získáme dvě rovnice 1. řádu:

$$
\dot{x}_1 = x_2
$$

$$
\dot{x}_2 = -\frac{g}{l} \sin(x_1) - \frac{b}{ml^2} x_2
$$

---

## 🧮 3. Stavový model (nelineární)

$$
\dot{\mathbf{x}} =
\begin{bmatrix}
\dot{x}_1 \\
\dot{x}_2
\end{bmatrix}
=
\begin{bmatrix}
x_2 \\
-\dfrac{g}{l} \sin(x_1) - \dfrac{b}{ml^2} x_2
\end{bmatrix}
$$

---

## 📤 4. Výstup (měřený úhel)

$$
y = x_1
$$

---

## ✅ Finální stavový model tlumeného kyvadla

$$
\dot{x}_1 = x_2
$$

$$
\dot{x}_2 = -\frac{g}{l} \sin(x_1) - \frac{b}{ml^2} x_2
$$

$$
y = x_1
$$


In [5]:
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

In [6]:
# Parametry systému
g = 9.81  # gravitační zrychlení [m/s^2]
m = 1.0  # hmotnost [kg]
l = 2.0  # délka kyvadla [m]
b = 1.0   # koeficient tlumení [N·s/m]

# Pravočlen diferenciální rovnice (stavový model)
def spring_system(state, t):
    x1, x2 = state  # x1 = úhel, x2 = úhlová rychlost
    dx1dt = x2
    dx2dt = - (g / l) * np.sin(x1) - (b / (m * l**2)) * x2
    return [dx1dt, dx2dt]

# Počáteční podmínky: theta = 30° (převedeno na radiány), omega = 0 rad/s
initial_state = [np.radians(30), 0.0]

# Časová osa pro simulaci (např. 0 až 10 s)
t = np.linspace(0, 4, 400)

# Integrace systému
solution = odeint(spring_system, initial_state, t)

# Výstup: solution je n x 2 pole (pozice a rychlost)
print("Výstup shape:", solution.shape)  # (1000, 2)

Výstup shape: (400, 2)


In [ ]:
# 🔧 Příprava animace kyvadla
%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

fig_of_plot, axes = plt.subplots(2, 1, constrained_layout=True, figsize=(7, 10))

# === PRVNÍ GRAF – Fyzický pohyb kyvadla ===
axes[0].set_xlim((-1.2, 1.2))
axes[0].set_ylim((-1.2, 0.2))
axes[0].set_title('Pohyb kyvadla')
axes[0].set_aspect('equal')
axes[0].grid()

# === DRUHÝ GRAF – Úhel a úhlová rychlost v čase ===
axes[1].set_xlim((0, len(solution)))
axes[1].set_ylim((-2.5, 2.5))
axes[1].set_title('Stavové proměnné kyvadla')
axes[1].grid()

# === Prázdné objekty ===
pendulum_line, = axes[0].plot([], [], 'b-', linewidth=2)           # tyč
pendulum_mass, = axes[0].plot([], [], 'ro', markersize=10)         # hmotný bod
angle_line, = axes[1].plot([], [], 'g-', label='úhel [rad]')       # x1
angular_velocity_line, = axes[1].plot([], [], 'r-', label='rychlost [rad/s]')  # x2
axes[1].legend()

# === Délka kyvadla ===
l = 1.0

# === Aktualizační funkce ===
def update_plot(i):
    if i < 2:
        return

    theta_vals = solution[:i, 0]
    omega_vals = solution[:i, 1]
    theta = theta_vals[-1]

    # Souřadnice konce kyvadla
    x = l * np.sin(theta)
    y = -l * np.cos(theta)

    # Aktualizace fyzikální animace
    pendulum_line.set_data([0, x], [0, y])
    pendulum_mass.set_data([x], [y])

    # Průběhy proměnných
    angle_line.set_data(np.arange(i), theta_vals)
    angular_velocity_line.set_data(np.arange(i), omega_vals)

# === Spuštění animace ===
ani = animation.FuncAnimation(fig_of_plot, update_plot, frames=len(solution), interval=10, repeat=False)

HTML(ani.to_jshtml())
